<a href="https://colab.research.google.com/github/fsternini/brats-task2/blob/development/brats_task2_launcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# First, verification of Python version is completed
!which python # should return /usr/local/bin/python
!python --version

/usr/local/bin/python
Python 3.10.12


In [2]:
# Google Drive is mounted, to reach all relevant data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Change the working directory reach the local repo

%cd /content/drive/MyDrive/Data Science/Github/brats-task2

/content/drive/MyDrive/Data Science/Github/brats-task2


# Nuova sezione

In [4]:
#!git clone https://github.com/fsternini/brats-task2.git

Cloning into 'brats-task2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 60 (delta 20), reused 45 (delta 12), pack-reused 0
Receiving objects: 100% (60/60), 12.37 KiB | 6.19 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [4]:
# Switch to the most updated branch
# Update the branch to the relevant one
!git switch development

Already on 'development'
Your branch is up to date with 'origin/development'.


In [5]:
# Collect the most updated version of the repo
!git pull

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 4), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (8/8), 1.73 KiB | 6.00 KiB/s, done.
From https://github.com/fsternini/brats-task2
   c20bffa..05f8565  development -> origin/development
Updating c20bffa..05f8565
Fast-forward
 brats_task2_launcher.ipynb | 107 ++++++++++++++++++++++++++++++++++++---------
 code/common/common.py      |  32 +++++++-------
 2 files changed, 103 insertions(+), 36 deletions(-)


In [6]:
# Install all requirements for the code
!pip install -r requirements.txt

In [7]:
import sys
sys.path.append('/content/drive/MyDrive/Data Science/Github/brats-task2/code')
import common.common

In [ ]:
from common.common import DatasetCreator

In [ ]:
dataset_creator = DatasetCreator('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset')

In [8]:
from common.common import Dataset
ds = Dataset('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/datasetdataset_structure.json')

In [12]:
ds.get_files_in_node("FLAIR")

['/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/test/00001/FLAIR/Image-100.dcm',
 '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/test/00001/FLAIR/Image-123.dcm',
 '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/test/00001/FLAIR/Image-119.dcm',
 '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/test/00001/FLAIR/Image-108.dcm',
 '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/test/00001/FLAIR/Image-10.dcm',
 '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/test/00001/FLAIR/Image-120.dcm',
 '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/test/00001/FLAIR/Image-117.dcm',
 '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classific

None
